In [1]:

from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33633?from=job%20search%20funnelmicr")
page_data = loader.load().pop().page_content
print(page_data)



Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsFIND A STOREBECOME A MEMBERSIGN UP FOR EMAILSEND US FEEDBACKSTUDENT DISCOUNTSGET HELPGET HELPOrder StatusDeliveryReturnsPayment OptionsC

In [4]:

from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': '', 'experience': '', 'skills': [], 'description': ''}]

In [13]:

type(json_res_dict)

NameError: name 'json_res_dict' is not defined

In [14]:

import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'my_portfolio.csv'

In [46]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [47]:
links = collection.query(query_texts=job.get('skills', ''), n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [48]:
job

{'Manager, State and Local, Government Affairs and Public Policy': {'experience': '',
  'skills': '',
  'description': ''},
 'Chip Package Signal and Power Integrity Engineer': {'experience': '',
  'skills': '',
  'description': ''},
 'Senior Financial Analyst, Supply Chain': {'experience': '',
  'skills': '',
  'description': ''},
 'Senior Security Engineer, Google Kubernetes Engine': {'experience': '',
  'skills': '',
  'description': ''},
 'Industry Architect, Health and Human Services, Public Sector': {'experience': '',
  'skills': '',
  'description': ''},
 'Health and Home Strategy Lead': {'experience': '',
  'skills': '',
  'description': ''},
 'Software Engineering Manager II, Site Reliability Engineering, Shopping': {'experience': '',
  'skills': '',
  'description': ''},
 'AI Sales Specialist': {'experience': '', 'skills': '', 'description': ''},
 'BigQuery SQL Translation and Lineage Senior Engineer': {'experience': '',
  'skills': '',
  'description': ''},
 'Staff Software 

In [52]:
# Assuming json_res is a list of dictionaries
for job in json_res:
    skills = job.get('skills', '')
    print(skills)


['Customer service', 'Product knowledge', 'Teamwork', 'Communication', 'Physical requirements: ability to bend, squat, reach, climb a ladder and stand for extended periods of time']


In [53]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Customer Experience with AtliQ's Retail Solutions

Dear Hiring Manager,

I came across the Retail Associate role at Nike and was impressed by the emphasis on exceptional customer service and teamwork. As a Business Development Executive at AtliQ, I believe our expertise in developing innovative retail solutions can help elevate the customer experience and streamline operations for Nike.

At AtliQ, we specialize in creating customized software solutions that cater to the unique needs of businesses like yours. Our team of experts has a proven track record of delivering scalable, efficient, and cost-effective solutions that drive growth and success.

In the retail sector, we've developed solutions that enhance customer engagement, improve inventory management, and optimize store operations. For instance, our Android TV portfolio showcases our capabilities in creating immersive and interactive experiences for customers. You can explore our portfolio here: https://example.c